<h2> Imports </h2>

In [5]:
import os
import pyodbc 
import matplotlib.pyplot as plt
import requests
import json 
import csv

<h2> Connect to SQL Server using pyodbc </h2>

In [17]:
server = 'SERVER NAME'
database = 'DATABASE NAME' 
driver = '{ODBC Driver 17 for SQL Server}'
conn_string = 'DRIVER='+driver+'; SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;'

In [3]:
def SQLServerConnect(conn_string): 
    '''Assumes conn_string is a string, establishes connection to SQL Server database
    '''
    cnxn = pyodbc.connect(conn_string)
    cnxn.setdecoding(pyodbc.SQL_CHAR, encoding='latin1')
    cnxn.setencoding('latin1')
    cursor = cnxn.cursor()  
    return cursor

In [ ]:
cursor = SQLServerConnect(conn_string) #Establish Connection

<h2> Build Queries for Visual Crossing API </h2>
    

In [6]:
key = '' # Visual Crossing API key
gameRec = [] # List of Game ID's
gameDate = [] # List of Game Dates * MUST BE IN 'yyyy-mm-dd' order *
gameLAT = [] # List of Game Latitudes
gameLONG = [] # List of Game Longitudes
queryList = [] # List of queries to send to Visual Crossing API 

In [ ]:
def buildVCAPIquery(lat, lon, dt, key): 
    '''Assumes lat, lon are coordinates, dt datetime as a string in yyyy-mm-dd format
        returns a Visual Crossing API Query for those coordinates at that time'''
    url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/%f,%f/%s?unitGroup=us&key=%s"%(lat,lon,dt, key)
    return url

In [ ]:
def buildVCAPIqueries(key, queryList, gameRec, gameDate, gameLAT, gameLONG): 
    '''Assumes key is Visual Crossing API key, 
       queryList a list, gameRec, gameDate, gameLAT, gameLONG lists of same length
       iterates using length of gameRec
       calls the buildVCAPIquery to add line to queryList'''
    i = 0
    while i < len(gameRec):
        qURL = buildVCAPIquery(gameLAT[i], gameLONG[i], gameDate[i], key)
        queryList.append(qURL)
        i += 1

<u><h3> There are multiple ways to build these queries </h3></u> 


<h4> 1. Via call to SQL Server DB using the latitude and longitude of the stadium on a home game basis. </h4>
<p>  There are two tables, MLBGames that has all home games from the MLB Season, and Stadium that has all the geo data for the stadium locations. Returns GameID, Date, Stadium Latitude, Stadium Longitude.     

In [7]:
def buildQueriesUsingSQL(key, cursor, gameRec, gameDate, gameLAT, gameLONG):
    '''Assumes key is a visual crossing API key, 
       cursor is a pyodbc cursor connection, 
       gameRec, gameDate, gameLAT, gameLONG lists
       Queries my SQL Database for ID, Date, Latitude, and Longitude
       Appends relevant data to each list,
       then calls buildVCAPIqueries function'''
    
    # construct an SQL query that returns the Gameid, datetime, lat, long from MLBGames and Stadiums
    insert_query = '''SELECT mlb.id, mlb.GameDateConverted, std.LAT, std.LONG
                        FROM Portfolio..MLBGames mlb
                        JOIN Portfolio..Stadiums std
                        ON mlb.Stadium = std.id
                        WHERE mlb.venue = 'Home'
                        ORDER BY mlb.GameDateConverted;'''
    cursor.execute(insert_query) # Execute the query

    for row in cursor: 
        gameRec.append(str(row[0])) # Build GameID list
        gameDate.append(row[1]) # Build GameDate list
        gameLAT.append(row[2]) # Build Latitude List
        gameLONG.append(row[3]) # Build Longitude List   
    
    buildVCAPIqueries(key, queryList, gameRec, gameDate, gameLAT, gameLONG)



<h4> 2. Via csv with rows structured 'GameID', 'Date', 'latitude', 'longitude'</h4>
<p> Important note for Date: Use single quotations and yyyy-mm-dd format, e.g. '2023-04-30'

In [15]:
def buildQueriesUsingCSV(filename, key, queryList, gameRec, gameDate, gameLAT, gameLONG): 
    '''Assumes filename is a csv with commas seperators and single quotes usage,
       key is Visual Crossing API key, 
       queryList is list of strings, 
       gameRec, gameDate, gameLAT, gameLONG are lists of same length
       appends relevant data to each list, then calls buildVCAPIqueries function'''
    with open(filename) as fp: 
        reader = csv.reader(fp, delimiter=',', quotechar="'")
        data_read = [row for row in reader]
    
    i = 0 
    for i in len(data_read): 
        gameRec.append(data_read[i][0])
        gameDate.append(data_read[i][1])
        gameLAT.append(data_read[i][2])
        gameLONG.append(data_read[i][3])
        i += 1
    
    buildVCAPIqueries(key, queryList, gameRec, gameDate, gameLAT, gameLONG)

    

1


<h4> 3. Build from Save Files </h4> 

<p> The last way to get ready for querying the VC API is the retrieve previous runs files and feed them back into the lists. If so, it is potentially harmful to run the above code instead of this code. Once you have constructed the list, you should then save your progress to txt files and run this code instead </p> 

In [ ]:
def buildFromFile(gameIDFileName, queryListFileName, CompleteListFileName, indexValueFileName): 
    '''Assumes 4 txt files already exists that were generated last time the program was run. 
       gameRec is the textFile with the remaining gameID's, 
       queryList is the txt file with the remaining queryList to run
       completeList is the txt file with all successful VC queries run 
       indexValue is the txt file with the current primary key index for the table in SQL
    '''
    gameRec = []
    f = open(gameIDFileName, 'r')
    for line in f: 
        gameRec.append(line)
    f.close()

    queryList = []
    f = open(queryListFileName, 'r')
    for line in f: 
        queryList.append(line)
    f.close()

    completeList = []
    f = open(CompleteListFileName, 'r')
    for line in f: 
        completeList.append(line)
    f.close()

    f = open(indexValueFileName, 'r')
    for line in f: 
        indexID = line
    f.close()

    return gameRec, queryList, completeList, indexID
    

In [9]:
if len(queryList) == 0 and os.path.isfile('queryList.txt') == False: 
    buildQueriesUsingSQL(key, cursor, gameRec, gameDate, gameLAT, gameLONG)

if os.path.isfile('queryList.txt'): 
    gameRec, queryList, completeList, indexID = buildFromFile('remainingGames.txt', 'queryList.txt', 'completedList.txt', 'indexvalue.txt')


<h2> Preparing the SQL Database for JSON returned from the API </h2> 

<p> The table has 430 columns. 1 is primary key, 1 is foreign key, 20 columns are daily averages, 408 are hourly data. There are 17 columns per hour. In order to build the table, I created a new table in SMSS called 'Weather'. I then built a function that generated an ADD call to the SQL database. </p>  

<h4> Building the first part of the insert query </h4>

In [ ]:
add_query = 'INSERT INTO Portfolio..Weather ('
res_query = 'ID, GameId, Date, tempmax, tempmin, temp, dew, humidity, precip, windgust, windspeed, winddir, pressure, cloudcover, visibility, solarRadiation, UVIndex, Severerisk, Sunrise, Sunset, Moonphase, conditions,'
for i in range(0,24): 
    temp = 'Hour' +str(i) +'temp'
    feelslike = 'Hour' +str(i) +'feelslike'
    humidity = 'Hour' +str(i) +'humidity'
    dew = 'Hour' +str(i) + 'dew'
    precip = 'Hour' +str(i) + 'precip'
    precipprob = 'Hour' +str(i) + 'precipprob'
    windgust = 'Hour' +str(i) + 'windgust'
    windspeed = 'Hour' +str(i) + 'windspeed'
    winddir = 'Hour' +str(i) + 'winddir'
    pressure = 'Hour' +str(i) + 'pressure'
    visibility = 'Hour' +str(i) + 'visibility'
    cloudcover = 'Hour' +str(i) + 'cloudcover'
    solarradiation = 'Hour' +str(i) + 'solarradiation'
    solarenergy = 'Hour' +str(i) + 'solarenergy'
    uvindex = 'Hour' +str(i) + 'uvindex'
    severerisk = 'Hour' +str(i) + 'severerisk'
    conditions = 'Hour' +str(i) + 'conditions'

    if i == 23: 
        res_query += (f'''{temp}, {feelslike}, {humidity}, {dew}, {precip}, {precipprob}, {windgust}, {windspeed}, {winddir}, 
        {pressure}, {visibility}, {cloudcover}, {solarradiation}, {solarenergy}, {uvindex}, {severerisk}, {conditions})''')
    else: 
        res_query += (f'''{temp}, {feelslike}, {humidity}, {dew}, {precip}, {precipprob}, {windgust}, {windspeed}, {winddir}, 
        {pressure}, {visibility}, {cloudcover}, {solarradiation}, {solarenergy}, {uvindex}, {severerisk}, {conditions},''')

part1_query = add_query + res_query

<h4> Making the VC API Call and Adding it to the Weather table </h4>


In [16]:
  
lim = 984 # Cost Limit. Each call is 24 records, you get 1000 free records a day. 24 * 41 is 984. Change to 24 * len(gameRec) if you want the whole thing
cost = 0 # keeps track of API call costs. 24 per call

while cost < lim: 
    # Connect to the database
    cursor = SQLServerConnect(conn_string)
    # Strip the queryList of newline characters if there are any
    queryList[0] = queryList[0].strip('\n')
    # Run Query on VC API using the top of queryList
    response = requests.get(queryList[0])

    # VERIFY THAT VC API RETURNED JSON DATA
    data = json.loads(response.text)
    #print(data)

    # PARSE JSON DATA INTO RELEVANT FIELDS
    identry = indexID
    game = (int(gameRec[0]))
    gameDate = "'"+ (data['days'][0]['datetime']) + "'"
    tempMax = (data['days'][0]['tempmax'])
    tempMin = (data['days'][0]['tempmin'])
    tempAVG =  (data['days'][0]['temp'])
    dewAVG =  (data['days'][0]['dew'])
    humidityAVG =  (data['days'][0]['humidity'])
    precipAVG =  (data['days'][0]['precip'])
    windgustAVG =  (data['days'][0]['windgust'])
    windspeedAVG =  (data['days'][0]['windspeed'])
    winddirAVG =  (data['days'][0]['winddir'])
    pressureAVG =  (data['days'][0]['pressure'])
    cloudcovAVG =  (data['days'][0]['cloudcover'])
    visibilityAVG =  (data['days'][0]['visibility'])
    solarRadAVG =  (data['days'][0]['solarradiation'])
    uvindAVG =  (data['days'][0]['uvindex'])
    sevriskAVG =  (data['days'][0]['severerisk'])
    sunriseAVG =  "'" + (data['days'][0]['sunrise']) + "'"
    sunsetAVG =  "'" + (data['days'][0]['sunset']) + "'"
    moonphaseAVG =  (data['days'][0]['moonphase'])
    conditionsAVG =  "'" + ((data['days'][0]['conditions'])) + "'"
    #get values for 24 days, save them to a dictionary 
    my_dict = {}
    for i in range(0,24): 
        
        #Variable names
        temp = 'temp'+str(i)
        feelslike = 'feelslike'+str(i)
        humidity = 'humidity'+str(i)
        dew = 'dew'+str(i)
        precip = 'precip'+str(i)
        precipprob = 'precipprob'+str(i)
        windgust = 'windgust'+str(i)
        windspeed = 'windspeed' + str(i)
        winddir = 'winddir' + str(i)
        pressure = 'pressure' + str(i)
        visibility = 'visibility' + str(i)
        cloudcover = 'cloudcover' + str(i)
        solarradiation = 'solarradiation' + str(i)
        solarenergy = 'solarenergy' + str(i)
        uvindex = 'uvindex' + str(i)
        severerisk = 'severerisk' + str(i)
        conditions = 'conditions' + str(i)

        #Append to dictionary
        my_dict[temp] = data['days'][0]['hours'][i]['temp']
        my_dict[feelslike] = data['days'][0]['hours'][i]['feelslike']
        my_dict[humidity] = data['days'][0]['hours'][i]['humidity']
        my_dict[dew] = data['days'][0]['hours'][i]['dew']
        my_dict[precip] = data['days'][0]['hours'][i]['precip']
        my_dict[precipprob] = data['days'][0]['hours'][i]['precipprob']
        my_dict[windgust] = data['days'][0]['hours'][i]['windgust']
        my_dict[windspeed] = data['days'][0]['hours'][i]['windspeed']
        my_dict[winddir] = data['days'][0]['hours'][i]['winddir']
        my_dict[pressure] = data['days'][0]['hours'][i]['pressure']
        my_dict[visibility] = data['days'][0]['hours'][i]['visibility']
        my_dict[cloudcover] = data['days'][0]['hours'][i]['cloudcover']
        my_dict[solarradiation] = data['days'][0]['hours'][i]['solarradiation']
        
        my_dict[uvindex] = data['days'][0]['hours'][i]['uvindex']
        my_dict[severerisk] = data['days'][0]['hours'][i]['severerisk']
        my_dict[conditions] = "'" + (data['days'][0]['hours'][i]['conditions']) + "'"

        if (data['days'][0]['hours'][i]['solarenergy'] == None): 
            my_dict[solarenergy] = 'NULL'
        else:     
            my_dict[solarenergy] = data['days'][0]['hours'][i]['solarenergy']

    # Build Query to add Field Values to SQL Table Weather using f format and my_dict holding all 24 hour data
    part2_query = f'''
    VALUES ({indexID},{game},{gameDate},{tempMax},{tempMin}, {tempAVG}, {dewAVG}, {humidityAVG}, {precipAVG}, {windgustAVG}, {windspeedAVG}, {winddirAVG}, {pressureAVG}, {cloudcovAVG}, {visibilityAVG}, {solarRadAVG}, {uvindAVG}, {sevriskAVG},{sunriseAVG}, {sunsetAVG}, {moonphaseAVG}, {conditionsAVG},
{my_dict['temp0']},{my_dict['feelslike0']}, {my_dict['humidity0']}, {my_dict['dew0']}, {my_dict['precip0']},{my_dict['precipprob0']}, {my_dict['windgust0']}, {my_dict['windspeed0']}, {my_dict['winddir0']}, {my_dict['pressure0']},{my_dict['visibility0']}, {my_dict['cloudcover0']}, {my_dict['solarradiation0']}, {my_dict['solarenergy0']}, {my_dict['uvindex0']},{my_dict['severerisk0']}, {my_dict['conditions0']},
{my_dict['temp1']},{my_dict['feelslike1']}, {my_dict['humidity1']}, {my_dict['dew1']}, {my_dict['precip1']},{my_dict['precipprob1']}, {my_dict['windgust1']}, {my_dict['windspeed1']}, {my_dict['winddir1']}, {my_dict['pressure1']},{my_dict['visibility1']}, {my_dict['cloudcover1']}, {my_dict['solarradiation1']}, {my_dict['solarenergy1']}, {my_dict['uvindex1']},{my_dict['severerisk1']}, {(my_dict['conditions1'])},
{my_dict['temp2']},{my_dict['feelslike2']}, {my_dict['humidity2']}, {my_dict['dew2']}, {my_dict['precip2']},{my_dict['precipprob2']}, {my_dict['windgust2']}, {my_dict['windspeed2']}, {my_dict['winddir2']}, {my_dict['pressure2']},{my_dict['visibility2']}, {my_dict['cloudcover2']}, {my_dict['solarradiation2']}, {my_dict['solarenergy2']}, {my_dict['uvindex2']},{my_dict['severerisk2']}, {my_dict['conditions2']},
{my_dict['temp3']},{my_dict['feelslike3']}, {my_dict['humidity3']}, {my_dict['dew3']}, {my_dict['precip3']},{my_dict['precipprob3']}, {my_dict['windgust3']}, {my_dict['windspeed3']}, {my_dict['winddir3']}, {my_dict['pressure3']},{my_dict['visibility3']}, {my_dict['cloudcover3']}, {my_dict['solarradiation3']}, {my_dict['solarenergy3']}, {my_dict['uvindex3']},{my_dict['severerisk3']}, {my_dict['conditions3']},
{my_dict['temp4']},{my_dict['feelslike4']}, {my_dict['humidity4']}, {my_dict['dew4']}, {my_dict['precip4']},{my_dict['precipprob4']}, {my_dict['windgust4']}, {my_dict['windspeed4']}, {my_dict['winddir4']}, {my_dict['pressure4']},{my_dict['visibility4']}, {my_dict['cloudcover4']}, {my_dict['solarradiation4']}, {my_dict['solarenergy4']}, {my_dict['uvindex4']},{my_dict['severerisk4']}, {my_dict['conditions4']},
{my_dict['temp5']},{my_dict['feelslike5']}, {my_dict['humidity5']}, {my_dict['dew5']}, {my_dict['precip5']},{my_dict['precipprob5']}, {my_dict['windgust5']}, {my_dict['windspeed5']}, {my_dict['winddir5']}, {my_dict['pressure5']},{my_dict['visibility5']}, {my_dict['cloudcover5']}, {my_dict['solarradiation5']}, {my_dict['solarenergy5']}, {my_dict['uvindex5']},{my_dict['severerisk5']}, {my_dict['conditions5']},
{my_dict['temp6']},{my_dict['feelslike6']}, {my_dict['humidity6']}, {my_dict['dew6']}, {my_dict['precip6']},{my_dict['precipprob6']}, {my_dict['windgust6']}, {my_dict['windspeed6']}, {my_dict['winddir6']}, {my_dict['pressure6']},{my_dict['visibility6']}, {my_dict['cloudcover6']}, {my_dict['solarradiation6']}, {my_dict['solarenergy6']}, {my_dict['uvindex6']},{my_dict['severerisk6']}, {my_dict['conditions6']},
{my_dict['temp7']},{my_dict['feelslike7']}, {my_dict['humidity7']}, {my_dict['dew7']}, {my_dict['precip7']},{my_dict['precipprob7']}, {my_dict['windgust7']}, {my_dict['windspeed7']}, {my_dict['winddir7']}, {my_dict['pressure7']},{my_dict['visibility7']}, {my_dict['cloudcover7']}, {my_dict['solarradiation7']}, {my_dict['solarenergy7']}, {my_dict['uvindex7']},{my_dict['severerisk7']}, {my_dict['conditions7']},
{my_dict['temp8']},{my_dict['feelslike8']}, {my_dict['humidity8']}, {my_dict['dew8']}, {my_dict['precip8']},{my_dict['precipprob8']}, {my_dict['windgust8']}, {my_dict['windspeed8']}, {my_dict['winddir8']}, {my_dict['pressure8']},{my_dict['visibility8']}, {my_dict['cloudcover8']}, {my_dict['solarradiation8']}, {my_dict['solarenergy8']}, {my_dict['uvindex8']},{my_dict['severerisk8']}, {my_dict['conditions8']},
{my_dict['temp9']},{my_dict['feelslike9']}, {my_dict['humidity9']}, {my_dict['dew9']}, {my_dict['precip9']},{my_dict['precipprob9']}, {my_dict['windgust9']}, {my_dict['windspeed9']}, {my_dict['winddir9']}, {my_dict['pressure9']},{my_dict['visibility9']}, {my_dict['cloudcover9']}, {my_dict['solarradiation9']}, {my_dict['solarenergy9']}, {my_dict['uvindex9']},{my_dict['severerisk9']}, {my_dict['conditions9']},
{my_dict['temp10']},{my_dict['feelslike10']}, {my_dict['humidity10']}, {my_dict['dew10']}, {my_dict['precip10']},{my_dict['precipprob10']}, {my_dict['windgust10']}, {my_dict['windspeed10']}, {my_dict['winddir10']}, {my_dict['pressure10']},{my_dict['visibility10']}, {my_dict['cloudcover10']}, {my_dict['solarradiation10']}, {my_dict['solarenergy10']}, {my_dict['uvindex10']},{my_dict['severerisk10']}, {my_dict['conditions10']},
{my_dict['temp11']},{my_dict['feelslike11']}, {my_dict['humidity11']}, {my_dict['dew11']}, {my_dict['precip11']},{my_dict['precipprob11']}, {my_dict['windgust11']}, {my_dict['windspeed11']}, {my_dict['winddir11']}, {my_dict['pressure11']},{my_dict['visibility11']}, {my_dict['cloudcover11']}, {my_dict['solarradiation11']}, {my_dict['solarenergy11']}, {my_dict['uvindex11']},{my_dict['severerisk11']}, {my_dict['conditions11']},
{my_dict['temp12']},{my_dict['feelslike12']}, {my_dict['humidity12']}, {my_dict['dew12']}, {my_dict['precip12']},{my_dict['precipprob12']}, {my_dict['windgust12']}, {my_dict['windspeed12']}, {my_dict['winddir12']}, {my_dict['pressure12']},{my_dict['visibility12']}, {my_dict['cloudcover12']}, {my_dict['solarradiation12']}, {my_dict['solarenergy12']}, {my_dict['uvindex12']},{my_dict['severerisk12']}, {my_dict['conditions12']},
{my_dict['temp13']},{my_dict['feelslike13']}, {my_dict['humidity13']}, {my_dict['dew13']}, {my_dict['precip13']},{my_dict['precipprob13']}, {my_dict['windgust13']}, {my_dict['windspeed13']}, {my_dict['winddir13']}, {my_dict['pressure13']},{my_dict['visibility13']}, {my_dict['cloudcover13']}, {my_dict['solarradiation13']}, {my_dict['solarenergy13']}, {my_dict['uvindex13']},{my_dict['severerisk13']}, {my_dict['conditions13']},
{my_dict['temp14']},{my_dict['feelslike14']}, {my_dict['humidity14']}, {my_dict['dew14']}, {my_dict['precip14']},{my_dict['precipprob14']}, {my_dict['windgust14']}, {my_dict['windspeed14']}, {my_dict['winddir14']}, {my_dict['pressure14']},{my_dict['visibility14']}, {my_dict['cloudcover14']}, {my_dict['solarradiation14']}, {my_dict['solarenergy14']}, {my_dict['uvindex14']},{my_dict['severerisk14']}, {my_dict['conditions14']},
{my_dict['temp15']},{my_dict['feelslike15']}, {my_dict['humidity15']}, {my_dict['dew15']}, {my_dict['precip15']},{my_dict['precipprob15']}, {my_dict['windgust15']}, {my_dict['windspeed15']}, {my_dict['winddir15']}, {my_dict['pressure15']},{my_dict['visibility15']}, {my_dict['cloudcover15']}, {my_dict['solarradiation15']}, {my_dict['solarenergy15']}, {my_dict['uvindex15']},{my_dict['severerisk15']}, {my_dict['conditions15']},
{my_dict['temp16']},{my_dict['feelslike16']}, {my_dict['humidity16']}, {my_dict['dew16']}, {my_dict['precip16']},{my_dict['precipprob16']}, {my_dict['windgust16']}, {my_dict['windspeed16']}, {my_dict['winddir16']}, {my_dict['pressure16']},{my_dict['visibility16']}, {my_dict['cloudcover16']}, {my_dict['solarradiation16']}, {my_dict['solarenergy16']}, {my_dict['uvindex16']},{my_dict['severerisk16']}, {my_dict['conditions16']},
{my_dict['temp17']},{my_dict['feelslike17']}, {my_dict['humidity17']}, {my_dict['dew17']}, {my_dict['precip17']},{my_dict['precipprob17']}, {my_dict['windgust17']}, {my_dict['windspeed17']}, {my_dict['winddir17']}, {my_dict['pressure17']},{my_dict['visibility17']}, {my_dict['cloudcover17']}, {my_dict['solarradiation17']}, {my_dict['solarenergy17']}, {my_dict['uvindex17']},{my_dict['severerisk17']}, {my_dict['conditions17']},
{my_dict['temp18']},{my_dict['feelslike18']}, {my_dict['humidity18']}, {my_dict['dew18']}, {my_dict['precip18']},{my_dict['precipprob18']}, {my_dict['windgust18']}, {my_dict['windspeed18']}, {my_dict['winddir18']}, {my_dict['pressure18']},{my_dict['visibility18']}, {my_dict['cloudcover18']}, {my_dict['solarradiation18']}, {my_dict['solarenergy18']}, {my_dict['uvindex18']},{my_dict['severerisk18']}, {my_dict['conditions18']},
{my_dict['temp19']},{my_dict['feelslike19']}, {my_dict['humidity19']}, {my_dict['dew19']}, {my_dict['precip19']},{my_dict['precipprob19']}, {my_dict['windgust19']}, {my_dict['windspeed19']}, {my_dict['winddir19']}, {my_dict['pressure19']},{my_dict['visibility19']}, {my_dict['cloudcover19']}, {my_dict['solarradiation19']}, {my_dict['solarenergy19']}, {my_dict['uvindex19']},{my_dict['severerisk19']}, {my_dict['conditions19']},
{my_dict['temp20']},{my_dict['feelslike20']}, {my_dict['humidity20']}, {my_dict['dew20']}, {my_dict['precip20']},{my_dict['precipprob20']}, {my_dict['windgust20']}, {my_dict['windspeed20']}, {my_dict['winddir20']}, {my_dict['pressure20']},{my_dict['visibility20']}, {my_dict['cloudcover20']}, {my_dict['solarradiation20']}, {my_dict['solarenergy20']}, {my_dict['uvindex20']},{my_dict['severerisk20']}, {my_dict['conditions20']},
{my_dict['temp21']},{my_dict['feelslike21']}, {my_dict['humidity21']}, {my_dict['dew21']}, {my_dict['precip21']},{my_dict['precipprob21']}, {my_dict['windgust21']}, {my_dict['windspeed21']}, {my_dict['winddir21']}, {my_dict['pressure21']},{my_dict['visibility21']}, {my_dict['cloudcover21']}, {my_dict['solarradiation21']}, {my_dict['solarenergy21']}, {my_dict['uvindex21']},{my_dict['severerisk21']}, {my_dict['conditions21']},
{my_dict['temp22']},{my_dict['feelslike22']}, {my_dict['humidity22']}, {my_dict['dew22']}, {my_dict['precip22']},{my_dict['precipprob22']}, {my_dict['windgust22']}, {my_dict['windspeed22']}, {my_dict['winddir22']}, {my_dict['pressure22']},{my_dict['visibility22']}, {my_dict['cloudcover22']}, {my_dict['solarradiation22']}, {my_dict['solarenergy22']}, {my_dict['uvindex22']},{my_dict['severerisk22']}, {my_dict['conditions22']},
{my_dict['temp23']},{my_dict['feelslike23']}, {my_dict['humidity23']}, {my_dict['dew23']}, {my_dict['precip23']},{my_dict['precipprob23']}, {my_dict['windgust23']}, {my_dict['windspeed23']}, {my_dict['winddir23']}, {my_dict['pressure23']},{my_dict['visibility23']}, {my_dict['cloudcover23']}, {my_dict['solarradiation23']}, {my_dict['solarenergy23']}, {my_dict['uvindex23']},{my_dict['severerisk23']}, {my_dict['conditions23']});'''

    insert_query = part1_query + part2_query
    #print(insert_query)
    # Execute Query
    cursor.execute(insert_query)
    cnxn.commit() # Commit the changes to the database
    # Close connection 
    cursor.close()
    cnxn.close()

    completeList.append(queryList[0]) #append query to completed list
    queryList.pop(0) # pop query from remaining list
    gameRec.pop(0) # Pop game record value
    # Calculate cost relative to daily limit. 41 a day basically
    cost += data['queryCost']
    indexID += 1
 
print('Made it to the end')
    
# Save the complete list and query list to files. 


NameError: name 'queryList' is not defined